In [6]:
# Cell 1: Load requirements.json and show as a table
import json
import pathlib
import pandas as pd

# Resolve repository root (searching upwards) so paths work regardless of kernel CWD)
def find_repo_root(start=pathlib.Path.cwd()):
    for cand in [start] + list(start.parents):
        # look for repo markers (adjust if your repo uses different files)
        if (cand / 'build.gradle').exists() or (cand / '.git').exists():
            return cand
    return pathlib.Path.cwd()
repo_root = find_repo_root()
p = repo_root / 'build' / 'results' / 'mission-control' / 'requirements.json'
if not p.exists():
    raise FileNotFoundError(f'Requirements file not found at {p} (repo_root={repo_root})')

with p.open() as f:
    data = json.load(f)

# Expecting SPARQL JSON results structure: results.bindings (flexible extraction)
bindings = data.get('results', {}).get('bindings', [])
rows = []

def short_from_uri(u):
    if not u:
        return ''
    if '#' in u:
        return u.split('#')[-1]
    return u.rstrip('/').split('/')[-1]

for b in bindings:
    # Name: find first uri in the binding and take the fragment after #
    name = None
    for v in b.values():
        if isinstance(v, dict) and v.get('type') == 'uri':
            name = short_from_uri(v.get('value'))
            break
    if not name:
        # fallback: try any value
        name = next((v.get('value') for v in b.values() if isinstance(v, dict) and v.get('value')), '')

    # Description and Expression: try to detect keys by name
    desc = None
    expr = None
    for k, v in b.items():
        kl = k.lower()
        if 'desc' in kl or 'description' in kl:
            desc = v.get('value')
        if 'expr' in kl or 'expression' in kl or 'logic' in kl:
            expr = v.get('value')
    rows.append({'Name': name, 'Description': desc, 'Expression': expr})

df = pd.DataFrame(rows)
# Display the DataFrame (Jupyter will render it as a nice table)
df

,Name,Description,Expression
0,R1,Real-Time Map,The system must display real-time fire and dro...
1,R4,Adaptive User Experience,The system must provide resilient user interfa...
2,R3,AI Warden Interface,"The system must enable direct, smooth communic..."
3,R2,Critical Alerts,The system must show real-time critical alerts...
4,R5,Historical Views,Support views of historical time series data


In [1]:
# Cell 2: Build an interactive graph of missions_capabilities.json
import json
import pathlib
import networkx as nx
import plotly.graph_objects as go

# Resolve repository root (searching upwards) so paths work regardless of kernel CWD)
def find_repo_root(start=pathlib.Path.cwd()):
    for cand in [start] + list(start.parents):
        if (cand / 'build.gradle').exists() or (cand / '.git').exists():
            return cand
    return pathlib.Path.cwd()
repo_root = find_repo_root()
p = repo_root / 'build' / 'results' / 'mission-control' / 'missions_capabilities.json'
if not p.exists():
    raise FileNotFoundError(f'Missions/capabilities file not found at {p} (repo_root={repo_root})')

with p.open() as f:
    data = json.load(f)

bindings = data.get('results', {}).get('bindings', [])
nodes = {}  # name -> description
edges = set()  # (a,b) tuples (unordered)

def short_from_uri(u):
    if not u:
        return None
    if '#' in u:
        return u.split('#')[-1]
    return u.rstrip('/').split('/')[-1]

for b in bindings:
    cap_uri = b.get('capabilityURI', {}).get('value')
    cap_desc = b.get('capabilityDescription', {}).get('value')
    sub_uri = b.get('subCapabilityURI', {}).get('value')
    sub_desc = b.get('subCapabilityDescription', {}).get('value')

    cap_name = short_from_uri(cap_uri)
    sub_name = short_from_uri(sub_uri) if sub_uri else None

    if cap_name:
        # Only set description if we don't already have one (prefer first non-empty)
        if cap_name not in nodes or not nodes[cap_name]:
            nodes[cap_name] = cap_desc
    if sub_name:
        if sub_name not in nodes or not nodes[sub_name]:
            nodes[sub_name] = sub_desc
        # add an undirected edge (dedupe by sorting)
        if cap_name and sub_name:
            edges.add(tuple(sorted((cap_name, sub_name))))

# Build the graph with networkx
G = nx.Graph()
for n, d in nodes.items():
    G.add_node(n, description=d)
for a, b in edges:
    if a and b:
        G.add_edge(a, b)

# Compute layout
pos = nx.spring_layout(G, seed=42) if len(G) > 0 else {}

# Edge traces
edge_x = []
edge_y = []
for u, v in G.edges():
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
edge_trace = go.Scatter(x=edge_x, y=edge_y, mode='lines', line=dict(width=1, color='#888'), hoverinfo='none')

# Node traces
node_x = []
node_y = []
hover_text = []
labels = []
for n in G.nodes():
    x, y = pos[n]
    node_x.append(x)
    node_y.append(y)
    desc = G.nodes[n].get('description') or ''
    hover_text.append(f'{n}<br>{desc}')
    labels.append(n)

node_trace = go.Scatter(
    x=node_x,
    y=node_y,
    mode='markers+text',
    text=labels,
    textposition='top center',
    hovertext=hover_text,
    hoverinfo='text',
    marker=dict(size=20, color='LightSkyBlue', line=dict(width=1, color='DarkSlateGrey'))
)

fig = go.Figure(data=[edge_trace, node_trace], layout=go.Layout(
    title='Mission Capabilities Graph',
    showlegend=False,
    hovermode='closest',
    margin=dict(b=20, l=5, r=5, t=40)
))

fig.show()